In [1]:
import numpy as np
import random
import sys
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [2]:
def randPair(s,e):
    return np.random.randint(s,e), np.random.randint(s,e)

class BoardPiece:

    def __init__(self, name, code, pos):
        self.name = name #name of the piece
        self.code = code #an ASCII character to display on the board
        self.pos = pos #2-tuple e.g. (1,4)

class BoardMask:

    def __init__(self, name, mask, code):
        self.name = name
        self.mask = mask
        self.code = code

    def get_positions(self): #returns tuple of arrays
        return np.nonzero(self.mask)

def zip_positions2d(positions): #positions is tuple of two arrays
    x,y = positions
    return list(zip(x,y))

class GridBoard:

    def __init__(self, size=4):
        self.size = size #Board dimensions, e.g. 4 x 4
        self.components = {} #name : board piece
        self.masks = {}

    def addPiece(self, name, code, pos=(0,0)):
        newPiece = BoardPiece(name, code, pos)
        self.components[name] = newPiece

    #basically a set of boundary elements
    def addMask(self, name, mask, code):
        #mask is a 2D-numpy array with 1s where the boundary elements are
        newMask = BoardMask(name, mask, code)
        self.masks[name] = newMask

    def movePiece(self, name, pos):
        move = True
        for _, mask in self.masks.items():
            if pos in zip_positions2d(mask.get_positions()):
                move = False
        if move:
            self.components[name].pos = pos

    def delPiece(self, name):
        del self.components['name']

    def render(self):
        dtype = '<U2'
        displ_board = np.zeros((self.size, self.size), dtype=dtype)
        displ_board[:] = ' '

        for name, piece in self.components.items():
            displ_board[piece.pos] = piece.code

        for name, mask in self.masks.items():
            displ_board[mask.get_positions()] = mask.code

        return displ_board

    def render_np(self):
        num_pieces = len(self.components) + len(self.masks)
        displ_board = np.zeros((num_pieces, self.size, self.size), dtype=np.uint8)
        layer = 0
        for name, piece in self.components.items():
            pos = (layer,) + piece.pos
            displ_board[pos] = 1
            layer += 1

        for name, mask in self.masks.items():
            x,y = self.masks['boundary'].get_positions()
            z = np.repeat(layer,len(x))
            a = (z,x,y)
            displ_board[a] = 1
            layer += 1
        return displ_board

def addTuple(a,b):
    return tuple([sum(x) for x in zip(a,b)])

In [3]:
class Gridworld:

    def __init__(self, size=4, mode='static'):
        if size >= 4:
            self.board = GridBoard(size=size)
        else:
            print("Minimum board size is 4. Initialized to size 4.")
            self.board = GridBoard(size=4)

        #Add pieces, positions will be updated later
        self.board.addPiece('Player','P',(0,0))
        self.board.addPiece('Goal','+',(0,3))
        self.board.addPiece('Pit','-',(0,3))
        self.board.addPiece('Wall','W',(2,3))

        if mode == 'static':
            self.initGridStatic()
        elif mode == 'player':
            self.initGridPlayer()
        else:
            self.initGridRand()

    #Initialize stationary grid, all items are placed deterministically
    def initGridStatic(self):
        #Setup static pieces
        self.board.components['Player'].pos = randPair(0,self.board.size) #Row, Column
        self.board.components['Goal'].pos = (3,0)
        self.board.components['Pit'].pos = (3,1)
        self.board.components['Wall'].pos = (3,2)
        
        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridStatic()


    #Check if board is initialized appropriately (no overlapping pieces)
    #also remove impossible-to-win boards
    def validateBoard(self):
        valid = True

        player = self.board.components['Player']
        goal = self.board.components['Goal']
        wall = self.board.components['Wall']
        pit = self.board.components['Pit']

        all_positions = [piece for name,piece in self.board.components.items()]
        all_positions = [player.pos, goal.pos, wall.pos, pit.pos]
        if len(all_positions) > len(set(all_positions)):
            return False

        corners = [(0,0),(0,self.board.size), (self.board.size,0), (self.board.size,self.board.size)]
        #if player is in corner, can it move? if goal is in corner, is it blocked?
        if player.pos in corners or goal.pos in corners:
            val_move_pl = [self.validateMove('Player', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            val_move_go = [self.validateMove('Goal', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            if 0 not in val_move_pl or 0 not in val_move_go:
                #print(self.display())
                #print("Invalid board. Re-initializing...")
                valid = False

        return valid

    #Initialize player in random location, but keep wall, goal and pit stationary
    def initGridPlayer(self):
        #height x width x depth (number of pieces)
        self.initGridStatic()
        #place player
        self.board.components['Player'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridPlayer()

    #Initialize grid so that goal, pit, wall, player are all randomly placed
    def initGridRand(self):
        #height x width x depth (number of pieces)
        self.board.components['Player'].pos = randPair(0,self.board.size)
        self.board.components['Goal'].pos = randPair(0,self.board.size)
        self.board.components['Pit'].pos = randPair(0,self.board.size)
        self.board.components['Wall'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridRand()

    def validateMove(self, piece, addpos=(0,0)):
        outcome = 0 #0 is valid, 1 invalid, 2 lost game
        pit = self.board.components['Pit'].pos
        wall = self.board.components['Wall'].pos
        new_pos = addTuple(self.board.components[piece].pos, addpos)
        if new_pos == wall:
            outcome = 1 #block move, player can't move to wall
        elif max(new_pos) > (self.board.size-1):    #if outside bounds of board
            outcome = 1
        elif min(new_pos) < 0: #if outside bounds
            outcome = 1
        elif new_pos == pit:
            outcome = 2

        return outcome

    def makeMove(self, action):
        #need to determine what object (if any) is in the new grid spot the player is moving to
        #actions in {u,d,l,r}
        def checkMove(addpos):
            if self.validateMove('Player', addpos) in [0,2]:
                new_pos = addTuple(self.board.components['Player'].pos, addpos)
                self.board.movePiece('Player', new_pos)

        if action == 'u': #up
            checkMove((-1,0))
        elif action == 'd': #down
            checkMove((1,0))
        elif action == 'l': #left
            checkMove((0,-1))
        elif action == 'r': #right
            checkMove((0,1))
        else:
            pass

    def reward(self):
        if (self.board.components['Player'].pos == self.board.components['Pit'].pos):
            return -10
        elif (self.board.components['Player'].pos == self.board.components['Goal'].pos):
            return 10
        else:
            return -1

    def display(self):
        return self.board.render()

In [4]:
action_set = {
    0: 'u',
    1: 'd',
    2: 'l',
    3: 'r',
}

In [5]:
"""
from2dto1d:
    Args: 
        pos(tuple):2d position of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(integer): 1d position of the objects(Player, Wall, Goal, Pit)in the gridworld
""" 
def from2dto1d(pos):
    if pos == '(0, 0)':
        return 0
    if pos == '(0, 1)':
        return 1
    if pos == '(0, 2)':
        return 2
    if pos == '(0, 3)':
        return 3
    if pos == '(1, 0)':
        return 4
    if pos == '(1, 1)':
        return 5
    if pos == '(1, 2)':
        return 6
    if pos == '(1, 3)':
        return 7
    if pos == '(2, 0)':
        return 8
    if pos == '(2, 1)':
        return 9
    if pos == '(2, 2)':
        return 10
    if pos == '(2, 3)':
        return 11
    if pos == '(3, 0)':
        return 12
    if pos == '(3, 1)':
        return 13
    if pos == '(3, 2)':
        return 14
    if pos == '(3, 3)':
        return 15

In [6]:
def from1dto2d(pos):
    if pos == 0:
        return (0, 0)
    if pos == 1:
        return (0, 1)
    if pos == 2:
        return (0, 2)
    if pos == 3:
        return (0, 3)
    if pos == 4:
        return (1, 0)
    if pos == 5:
        return (1, 1)
    if pos == 6:
        return (1, 2)
    if pos == 7:
        return (1, 3)
    if pos == 8:
        return (2, 0)
    if pos == 9:
        return (2, 1)
    if pos == 10:
        return (2, 2)
    if pos == 11:
        return (2, 3)
    if pos == 12:
        return (3, 0)
    if pos == 13:
        return (3, 1)
    if pos == 14:
        return (3, 2)
    if pos == 15:
        return (3, 3)

In [7]:
"""
from_num_to_one_hot_encode:
    Args: 
        num(int): number from 0 to 15 representing the state of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(tuple): encoded binary code with the size of 16-bit 
"""
def from_num_to_one_hot_encode(num):
    en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
    if num == 0:
        en0 = 1
    if num == 1:
        en1 = 1
    if num == 2:
        en2 = 1
    if num == 3:
        en3 = 1
    if num == 4:
        en4 = 1
    if num == 5:
        en5 = 1
    if num == 6:
        en6 = 1
    if num == 7:
        en7 = 1
    if num == 8:
        en8 = 1
    if num == 9:
        en9 = 1
    if num == 10:
        en10 = 1
    if num == 11:
        en11 = 1
    if num == 12:
        en12 = 1
    if num == 13:
        en13 = 1
    if num == 14:
        en14 = 1
    if num == 15:
        en15 = 1
    return en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15

In [8]:
df = pd.read_csv("Bellman_Operator.csv")
df

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Q1_Next,Reward_1_Next,Q2_Next,Reward_2_Next,Q3_Next,Reward_3_Next,Q4_Next,Reward_4_Next,Max_Q_Next,Reward_Max_Next,Bellman_Operator
0,0,0,2,1,0,-0.356023,-10,-10.000000,-100,2.647438,-1,-10.000000,-100,-10.000000,-100,2.647438,-1,1.382694
1,1,2,2,1,0,9.709291,10,-10.000000,-100,7.422089,-1,-10.000000,-100,7.962252,-1,7.962252,-1,10.000000
2,2,3,2,1,0,7.962252,-1,-10.000000,-100,6.114790,-1,9.709291,10,-10.000000,-100,9.709291,10,10.000000
3,3,4,2,1,0,2.647438,-1,-0.356023,-10,2.080958,-1,-10.000000,-100,6.384776,-1,6.384776,-1,4.746299
4,4,5,2,1,0,6.384776,-1,-10.000000,-100,3.147437,-1,2.647438,-1,7.422089,-1,7.422089,-1,5.679880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50395,50395,8,13,14,15,6.419645,-1,4.150536,-1,10.481168,-1,-10.000000,-100,6.489353,-1,10.481168,-1,8.433051
50396,50396,13,13,14,15,9.970391,10,6.489353,-1,-10.000000,-100,10.481168,-1,-10.000000,-100,10.481168,-1,10.000000
50397,50397,9,13,14,15,6.489353,-1,3.969529,-1,9.970391,10,6.419645,-1,6.954754,-1,9.970391,10,10.000000
50398,50398,12,13,14,15,10.481168,-1,6.419645,-1,-10.000000,-100,-10.000000,-100,9.970391,10,9.970391,10,10.000000


In [10]:
x = df[["Player"]]
y = OneHotEncoder().fit_transform(x).toarray()
dataset = pd.DataFrame({'Encode_0': y[:, 0], 'Encode_1': y[:, 1], 'Encode_2': y[:, 2], 'Encode_3': y[:, 3], 'Encode_4': y[:, 4], 'Encode_5': y[:, 5], 'Encode_6': y[:, 6], 'Encode_7': y[:, 7], 'Encode_8': y[:, 8], 'Encode_9': y[:, 9], 'Encode_10': y[:, 10], 'Encode_11': y[:, 11], 'Encode_12': y[:, 12], 'Encode_13': y[:, 13], 'Encode_14': y[:, 14], 'Encode_15': y[:, 15]})
df['Encode_0'] = dataset["Encode_0"]
df['Encode_1'] = dataset["Encode_1"]
df['Encode_2'] = dataset["Encode_2"]
df['Encode_3'] = dataset["Encode_3"]
df['Encode_4'] = dataset["Encode_4"]
df['Encode_5'] = dataset["Encode_5"]
df['Encode_6'] = dataset["Encode_6"]
df['Encode_7'] = dataset["Encode_7"]
df['Encode_8'] = dataset["Encode_8"]
df['Encode_9'] = dataset["Encode_9"]
df['Encode_10'] = dataset["Encode_10"]
df['Encode_11'] = dataset["Encode_11"]
df['Encode_12'] = dataset["Encode_12"]
df['Encode_13'] = dataset["Encode_13"]
df['Encode_14'] = dataset["Encode_14"]
df['Encode_15'] = dataset["Encode_15"]

In [11]:
df

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Q1_Next,Reward_1_Next,Q2_Next,...,Encode_6,Encode_7,Encode_8,Encode_9,Encode_10,Encode_11,Encode_12,Encode_13,Encode_14,Encode_15
0,0,0,2,1,0,-0.356023,-10,-10.000000,-100,2.647438,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,2,1,0,9.709291,10,-10.000000,-100,7.422089,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3,2,1,0,7.962252,-1,-10.000000,-100,6.114790,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4,2,1,0,2.647438,-1,-0.356023,-10,2.080958,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,2,1,0,6.384776,-1,-10.000000,-100,3.147437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50395,50395,8,13,14,15,6.419645,-1,4.150536,-1,10.481168,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50396,50396,13,13,14,15,9.970391,10,6.489353,-1,-10.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
50397,50397,9,13,14,15,6.489353,-1,3.969529,-1,9.970391,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
50398,50398,12,13,14,15,10.481168,-1,6.419645,-1,-10.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [16]:
no_dup_df = df
no_dup_df

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Q1_Next,Reward_1_Next,Q2_Next,...,Encode_6,Encode_7,Encode_8,Encode_9,Encode_10,Encode_11,Encode_12,Encode_13,Encode_14,Encode_15
0,0,0,2,1,0,-0.356023,-10,-10.000000,-100,2.647438,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,2,1,0,9.709291,10,-10.000000,-100,7.422089,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3,2,1,0,7.962252,-1,-10.000000,-100,6.114790,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4,2,1,0,2.647438,-1,-0.356023,-10,2.080958,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,2,1,0,6.384776,-1,-10.000000,-100,3.147437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50395,50395,8,13,14,15,6.419645,-1,4.150536,-1,10.481168,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50396,50396,13,13,14,15,9.970391,10,6.489353,-1,-10.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
50397,50397,9,13,14,15,6.489353,-1,3.969529,-1,9.970391,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
50398,50398,12,13,14,15,10.481168,-1,6.419645,-1,-10.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [47]:
counter = 0

h = 0
w, h = 17, 300000
matrix = [[0 for x in range(w)] for y in range(h)] 

for i in range (0, no_dup_df.shape[0]):
    game = Gridworld(size=4, mode='random')
    game.board.components['Player'].pos = from1dto2d(no_dup_df["Player"][i])
    game.board.components['Goal'].pos = from1dto2d(no_dup_df["Goal"][i])
    game.board.components['Wall'].pos = from1dto2d(no_dup_df["Wall"][i])
    game.board.components['Pit'].pos = from1dto2d(no_dup_df["Pit"][i])
    
    q_value = []
    reward_arr = []
    
    pi = from2dto1d((str(game.board.components['Player'].pos)))
    
    goal = int(no_dup_df["Goal"][i])
    pit = int(no_dup_df["Pit"][i])
    wall = int(no_dup_df["Wall"][i])
    
    matrix[counter][0] = pi
    matrix[counter][1] = no_dup_df["Goal"][i]
    matrix[counter][2] = no_dup_df["Wall"][i]
    matrix[counter][3] = no_dup_df["Pit"][i]
    matrix[counter][4] = no_dup_df["Q_Current"][i]
    
    if no_dup_df["Reward_Current"][i] == 10:
        matrix[counter][5] = 0.75
    if no_dup_df["Reward_Current"][i] == -1:
        matrix[counter][5] = 0.62
    if no_dup_df["Reward_Current"][i] == -10:
        matrix[counter][5] = 0.5
        
    game.makeMove('u')
    p = from2dto1d((str(game.board.components['Player'].pos)))

    if p == pi:
        matrix[counter][6] = -10
        q_value.append(-10)
        matrix[counter][7] = 0
        reward_arr.append(0)
    else:
        matrix[counter][6] = no_dup_df[(no_dup_df["Player"] == int(p)) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_Current'].values[0]
        q_value.append(no_dup_df[(no_dup_df["Player"] == int(p)) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_Current'].values[0])
        reward = game.reward()
        if reward == 10:
            reward = 0.75
        if reward == -1:
            reward = 0.62
        if reward == -10:
            reward = 0.5
        matrix[counter][7] = reward
        reward_arr.append(reward)
        game.makeMove('d')
        
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('d')
    p = from2dto1d((str(game.board.components['Player'].pos)))
            
    if p == pi:
        matrix[counter][8] = -10
        q_value.append(-10)
        matrix[counter][9] = 0
        reward_arr.append(0)
    else:
        matrix[counter][8] = no_dup_df[(no_dup_df["Player"] == p) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_Current'].values[0]
        q_value.append(no_dup_df[(no_dup_df["Player"] == p) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_Current'].values[0])
        reward = game.reward()
        if reward == 10:
            reward = 0.75
        if reward == -1:
            reward = 0.62
        if reward == -10:
            reward = 0.5
        matrix[counter][9] = reward
        reward_arr.append(reward)
        game.makeMove('u')
        
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('l')
    p = from2dto1d((str(game.board.components['Player'].pos)))
            
    if p == pi:
        matrix[counter][10] = -10
        q_value.append(-10)
        matrix[counter][11] = 0
        reward_arr.append(0)
    else:
        matrix[counter][10] = no_dup_df[(no_dup_df["Player"] == p) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_Current'].values[0]
        q_value.append(no_dup_df[(no_dup_df["Player"] == p) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_Current'].values[0])
        reward = game.reward()
        if reward == 10:
            reward = 0.75
        if reward == -1:
            reward = 0.62
        if reward == -10:
            reward = 0.5
        matrix[counter][11] = reward
        reward_arr.append(reward)
        game.makeMove('r')
        
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))       
    game.makeMove('r')
    p = from2dto1d((str(game.board.components['Player'].pos)))
            
    if p == pi:
        matrix[counter][12] = -10
        q_value.append(-10)
        matrix[counter][13] = 0
        reward_arr.append(0)
    else:
        matrix[counter][12] = no_dup_df[(no_dup_df["Player"] == p) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_Current'].values[0]
        q_value.append(no_dup_df[(no_dup_df["Player"] == p) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_Current'].values[0])
        reward = game.reward()
        if reward == 10:
            reward = 0.75
        if reward == -1:
            reward = 0.62
        if reward == -10:
            reward = 0.5
        matrix[counter][13] = reward
        reward_arr.append(reward)
        game.makeMove('l')
        
    
    max_qvalue = q_value[np.argmax(q_value)]
    matrix[counter][14] = max_qvalue
    reward_max = reward_arr[np.argmax(q_value)]
    matrix[counter][15] = reward_arr[np.argmax(q_value)]
    bellman = max_qvalue*0.9 + reward_max
    if pi == goal or bellman > 10:
        matrix[counter][16] = 10
    else:
        matrix[counter][16] = bellman
    counter += 1
    
    
    

In [48]:
import pandas as pd
column_names = ["Player","Goal", "Wall", "Pit", "Q_Current", "Reward_Current", "Q1_Next", "Reward_1_Next", 
                "Q2_Next", "Reward_2_Next", "Q3_Next", "Reward_3_Next", "Q4_Next", "Reward_4_Next", 
                "Max_Q_Next", "Reward_Max_Next", "Bellman_Operator"
                ]
bell_df = pd.DataFrame(matrix, columns = column_names)
bell_df = bell_df.loc[(bell_df != 0).any(axis=1)]

In [49]:
bell_df

,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Q1_Next,Reward_1_Next,Q2_Next,Reward_2_Next,Q3_Next,Reward_3_Next,Q4_Next,Reward_4_Next,Max_Q_Next,Reward_Max_Next,Bellman_Operator
0,0,2,1,0,-0.356023,0.50,-10.000000,0.00,2.647438,0.62,-10.000000,0.00,-10.000000,0.00,2.647438,0.62,3.002694
1,2,2,1,0,9.709291,0.75,-10.000000,0.00,7.422089,0.62,-10.000000,0.00,7.962252,0.62,7.962252,0.62,10.000000
2,3,2,1,0,7.962252,0.62,-10.000000,0.00,6.114790,0.62,9.709291,0.75,-10.000000,0.00,9.709291,0.75,9.488361
3,4,2,1,0,2.647438,0.62,-0.356023,0.50,2.080958,0.62,-10.000000,0.00,6.384776,0.62,6.384776,0.62,6.366299
4,5,2,1,0,6.384776,0.62,-10.000000,0.00,3.147437,0.62,2.647438,0.62,7.422089,0.62,7.422089,0.62,7.299880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50395,8,13,14,15,6.419645,0.62,4.150536,0.62,10.481168,0.62,-10.000000,0.00,6.489353,0.62,10.481168,0.62,10.000000
50396,13,13,14,15,9.970391,0.75,6.489353,0.62,-10.000000,0.00,10.481168,0.62,-10.000000,0.00,10.481168,0.62,10.000000
50397,9,13,14,15,6.489353,0.62,3.969529,0.62,9.970391,0.75,6.419645,0.62,6.954754,0.62,9.970391,0.75,9.723352
50398,12,13,14,15,10.481168,0.62,6.419645,0.62,-10.000000,0.00,-10.000000,0.00,9.970391,0.75,9.970391,0.75,9.723352


In [50]:
bell_df.to_csv('Reward_Changes_2.csv')

In [51]:
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)

In [52]:
X = bell_df[['Player','Pit', 'Goal', 'Wall']]
y = bell_df[['Bellman_Operator']]
mlp.fit(X.values,y.values.ravel())

MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [53]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [54]:
def mlp_test_model(model_b, experience, mode='static', display=True):
    global counter
    i = 0
    test_game = Gridworld(mode=mode)
    if display:
        print("Initial State:")
        print(test_game.display())
    status = 1
    j = 0
    while(status == 1): #A
        p_curr = from2dto1d((str(test_game.board.components['Player'].pos)))
        pi = from2dto1d((str(test_game.board.components['Pit'].pos)))
        g = from2dto1d((str(test_game.board.components['Goal'].pos)))
        w = from2dto1d((str(test_game.board.components['Wall'].pos)))

        q_value_next = []

        test_game.makeMove('u')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        
        if p_curr != p:
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('d')
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('d')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        
        if p_curr != p:
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('u')
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('l')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        

        if p_curr != p:
            test_game.makeMove('r')
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('r')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        

        if p_curr != p:
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('l')
        else:
            q_value_next.append(-10)
        
#         print(q_value_next)
        
        
        action_ = np.argmax(q_value_next)
#         print(action_)
        action = action_set[action_]
        
        experience[counter][0] = p_curr
        experience[counter][1] = pi
        experience[counter][2] = g
        experience[counter][3] = w
        experience[counter][4] = action_
        
    
        if display:
            print('Move #: %s; Taking action: %s' % (i, action))
        test_game.makeMove(action)
        p_next = from2dto1d((str(test_game.board.components['Player'].pos)))
        experience[counter][5] = p_next
        
        j += 1
        if display:
            print(test_game.display())
        reward = test_game.reward()
        experience[counter][6] = reward
        if reward > -1:
            experience[counter][7] = True
        else:
            experience[counter][7] = False
        experience[counter][8] =  q_value_next[action_]
        experience[counter][9] = q_value_next[0]
        experience[counter][10] = q_value_next[1]
        experience[counter][11] = q_value_next[2]
        experience[counter][12] = q_value_next[3]
        if reward != -1:
            if reward > 0:
                status = 2
                if display:
                    print("Game won! Reward: %s" % (reward,))
            else:
                status = 0
                if display:
                    print("Game LOST. Reward: %s" % (reward,))
        counter += 1
        i += 1
        if (i > 15):
            if display:
                print("Game lost; too many moves.")
            break
    
    win = True if status == 2 else False
    return win

In [56]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 900
0.9
0.9
Games played: 1000, # of wins: 914
0.914
1.814
Games played: 1000, # of wins: 896
0.896
2.71
Games played: 1000, # of wins: 893
0.893
3.6029999999999998
Games played: 1000, # of wins: 896
0.896
4.499
Games played: 1000, # of wins: 888
0.888
5.387
Games played: 1000, # of wins: 888
0.888
6.2749999999999995
Games played: 1000, # of wins: 884
0.884
7.159
Games played: 1000, # of wins: 893
0.893
8.052
Games played: 1000, # of wins: 885
0.885
8.937
Win percentage: 89.36999999999999%


In [40]:
df

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Q1_Next,Reward_1_Next,Q2_Next,...,Encode_6,Encode_7,Encode_8,Encode_9,Encode_10,Encode_11,Encode_12,Encode_13,Encode_14,Encode_15
0,0,0,2,1,0,-0.356023,-10,-10.000000,-100,2.647438,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,2,1,0,9.709291,10,-10.000000,-100,7.422089,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3,2,1,0,7.962252,-1,-10.000000,-100,6.114790,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4,2,1,0,2.647438,-1,-0.356023,-10,2.080958,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,2,1,0,6.384776,-1,-10.000000,-100,3.147437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50395,50395,8,13,14,15,6.419645,-1,4.150536,-1,10.481168,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50396,50396,13,13,14,15,9.970391,10,6.489353,-1,-10.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
50397,50397,9,13,14,15,6.489353,-1,3.969529,-1,9.970391,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
50398,50398,12,13,14,15,10.481168,-1,6.419645,-1,-10.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [41]:
df["Reward_Current"]

0       -10
1        10
2        -1
3        -1
4        -1
         ..
50395    -1
50396    10
50397    -1
50398    -1
50399    -1
Name: Reward_Current, Length: 50400, dtype: int64

In [43]:
array

array([-10,  10,  -1, ...,  -1,  -1,  -1], dtype=int64)

In [45]:
import statistics as st
array = df['Reward_Current'].to_numpy()
stdev = round(st.stdev(array),2)
mean = round(st.mean(array),2)
print(array)
print("stdev:",stdev)
print("mean", mean)

[-10  10  -1 ...  -1  -1  -1]
stdev: 3.61
mean 0


In [1]:
df

NameError: name 'df' is not defined